In [1]:
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
from tasho.OCPGenerator import OCPGenerator
from robotshyu import Robot as rob
from time import sleep
import numpy as np
import casadi as cs
import pybullet as p
import tasho
import robotshyu
import os

pybullet build time: May 20 2022 19:44:17


In [2]:
# Options for example codes
frame_enable = True

# Ex2) P2P Object picking MPC: MMO-500

In [39]:
robot = rob.Robot("mmo_500_ppr")
link_name = 9
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)

# Define initial conditions of the robot
q0_val = [-1.0, -0.523598, 1.5709, 3.14, 0, -0.523598, 0.0, 0.0, 0.0]
qd0_val = [0] * robot.ndof

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, rot_tol=1e-3)
# task_P2P.write_task_graph("MMO500_P2P_Task4.svg")

Loading robot params from json ...
Loaded 9-DoF robot: mmo_500_ppr


In [40]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_mmo_500_ppr'])
task_P2P.remove_terminal_constraints('rot_con_pose_9_mmo_500_ppr_vs_goal',
                                    'trans_con_pose_9_mmo_500_ppr_vs_goal')
# task_P2P.write_task_graph("MMO500_P2P_Task5.svg")

In [41]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_mmo_500_ppr']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_9_mmo_500_ppr_vs_goal'], 1e0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_9_mmo_500_ppr_vs_goal'], 8e-1), 
                            Regularization(task_P2P.variables['qd_mmo_500_ppr'], 3e-2),
                            Regularization(task_P2P.variables['qdd_mmo_500_ppr'],1e-2))

# task_P2P.write_task_graph("MMO500_P2P_Task6.svg")

In [42]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
mpc_options = default_mpc_options.get_default_mpc_options()

tc.ocp_solver = mpc_options["ipopt_init"]["solver_name"]
tc.ocp_options = mpc_options["ipopt_init"]["options"]
tc.mpc_solver = mpc_options["ipopt_lbfgs"]["solver_name"]
tc.mpc_options =mpc_options["ipopt_lbfgs"]["options"]
tc.set_ocp_solver(tc.ocp_solver, tc.ocp_options)

################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_mmo_500_ppr'].x
qd = pOCP.stage_tasks[0].variables['qd_mmo_500_ppr'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_mmo_500_ppr'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_mmo_500_ppr'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_mmo_500_ppr'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_mmo_500_ppr", "var":"qd_mmo_500_ppr", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_9_mmo_500_ppr_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

# os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
destroy semaphore
Thread TERMINATED
semaphore destroyed
destroy main semaphore
main semaphore destroyed
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 245.00us ( 17.50us) 242.76us ( 17.34us)        14
       nlp_g  | 102.00us (  7.29us)  99.19us (  7.08us)        14
    nlp_grad  |  78.00us ( 78.00us)  77.36us ( 77.36us)         1
  nlp_grad_f  | 649.00us ( 43.27us) 650.23us ( 43.35us)        15
  nlp_hess_l  |   3.49ms (268.31us)   3.49ms (268.49us)        13
   nlp_jac_g  | 218.00us ( 14.53us) 218.96us ( 14.60us)        15
       total  |   9.78ms (  9.78ms)   9.77ms (  9.77ms)         1


In [43]:
dir_casadi_func = "casadi_dir"
os.makedirs("./"+dir_casadi_func, exist_ok=True)
vars_db = tc.generate_MPC_component("./"+dir_casadi_func+"/", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti3_p_2[12:15]-fk_T(opti3_x_1){9}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti3_p_2[12:15]-fk_T(opti3_x_1){9}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [44]:
MPC_component = MPC("mmo_500_ppr_obj_pickup", "./"+dir_casadi_func+"/"+ tc.name + ".json")

In [45]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position,orientation,robot_pkg=robot)

if frame_enable:
    frameIDs = [0]*horizon_size
    for i in range(horizon_size):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_green.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")

# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 13, 14, 15, 16, 17, 18]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")

obj.resetJointState(robotID,joint_indices,q0_val)
if frame_enable==True:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0_val])

obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

q_log = []
q_dot_log = []
predicted_pos_log = []
q_pred = [0]*horizon_size

In [46]:
# reset cube velocity
p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# p.resetBaseVelocity(cubeID, linearVelocity=[-1, 0, 0])
# p.resetBaseVelocity(cubeID, linearVelocity=[1, 0, 0])

In [47]:
# Execute the MPC loop
for i in range(horizon_size * 10):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_mmo_500_ppr"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_mmo_500_ppr"]["val"] = qd_now

    # the position of the target object (cube)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)

    predicted_pos = lin_pos
    predicted_pos_log.append(predicted_pos.full())
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 0, 1, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([-1, 0, 0, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_mmo_500_ppr"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()
    if frame_enable==True:
        sol = MPC_component.res_vals
        for i in range(horizon_size):
            q_pred[i]=sol[robot.ndof*i:robot.ndof*i+robot.ndof].full()
        obj.resetMultiJointState(frameIDs, joint_indices, q_pred)

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_mmo_500_ppr"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_mmo_500_ppr"]["val"] * t_mpc).full()
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)
#     sleep(0.1)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, -0.2, 0.41]
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 362

----------- MPC execution -----------
Predicted position of cube [0.4999997744355, 0.18839929069618, 0.27248968087641]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 282.00us ( 23.50us) 282.84us ( 23.57us)        12
       nlp_g  | 117.00us (  9.75us) 115.53us (  9.63us)        12
    nlp_grad  |  98.00us ( 98.00us)  98.49us ( 98.49us)         1
  nlp_grad_f  | 742.00us ( 57.08us) 746.26us ( 57.40us)        13
  nlp_hess_l  |  16.57ms (  1.51ms)   3.58ms (325.51us)        11
   nlp_jac_g  | 246.00us ( 18.92us) 246.03us ( 18.93us)        13
       total  |  50.90ms ( 50.90ms)  12.79ms ( 12.79ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999975828779, 0.21160809869397, 0.27248985054166]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.15ms ( 95.83us) 279.56us ( 23.30us)        12
       nlp_g  | 121.00us ( 10.08us) 119.89us (  9.99us)        12
    nlp_grad  |  99.00us ( 99.00us)

----------- MPC execution -----------
Predicted position of cube [0.49999541209609, 0.42817205165349, 0.084989534531034]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 248.00us ( 22.55us) 246.91us ( 22.45us)        11
       nlp_g  | 122.00us ( 11.09us) 121.88us ( 11.08us)        11
    nlp_grad  |  96.00us ( 96.00us)  95.67us ( 95.67us)         1
  nlp_grad_f  | 663.00us ( 55.25us) 667.92us ( 55.66us)        12
  nlp_hess_l  |  16.74ms (  1.67ms)   3.39ms (339.45us)        10
   nlp_jac_g  | 239.00us ( 19.92us) 237.95us ( 19.83us)        12
       total  |  46.38ms ( 46.38ms)  11.05ms ( 11.05ms)         1
----------- MPC execution -----------
Predicted position of cube [0.4999952311193, 0.4320436697907, 0.08498982434801]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 448.00us ( 40.73us) 254.44us ( 23.13us)        11
       nlp_g  |   5.68ms (516.09us) 106.71us (  9.70us)        11
    nlp_grad  |   2.22ms (  2.22ms)

----------- MPC execution -----------
Predicted position of cube [0.49999339210657, 0.43634782581684, 0.084988353661209]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 255.00us ( 23.18us) 255.72us ( 23.25us)        11
       nlp_g  | 112.00us ( 10.18us) 113.66us ( 10.33us)        11
    nlp_grad  |  97.00us ( 97.00us)  96.87us ( 96.87us)         1
  nlp_grad_f  | 658.00us ( 54.83us) 656.81us ( 54.73us)        12
  nlp_hess_l  |   5.71ms (570.80us)   3.36ms (335.54us)        10
   nlp_jac_g  | 240.00us ( 20.00us) 239.39us ( 19.95us)        12
       total  |  43.42ms ( 43.42ms)  11.04ms ( 11.04ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999339210658, 0.43634782582234, 0.084988353655717]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.63ms (148.27us) 254.60us ( 23.15us)        11
       nlp_g  |   8.20ms (745.64us) 111.21us ( 10.11us)        11
    nlp_grad  |  95.00us ( 95.00

----------- MPC execution -----------
Predicted position of cube [0.49999339210659, 0.43634782582551, 0.08498835365255]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 253.00us ( 23.00us) 253.63us ( 23.06us)        11
       nlp_g  | 102.00us (  9.27us) 101.02us (  9.18us)        11
    nlp_grad  |  97.00us ( 97.00us)  97.35us ( 97.35us)         1
  nlp_grad_f  | 655.00us ( 54.58us) 655.31us ( 54.61us)        12
  nlp_hess_l  |   3.38ms (337.80us)   3.39ms (338.63us)        10
   nlp_jac_g  | 238.00us ( 19.83us) 238.38us ( 19.87us)        12
       total  |  26.84ms ( 26.84ms)  10.85ms ( 10.85ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999339210659, 0.43634782582551, 0.08498835365255]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 238.00us ( 21.64us) 239.76us ( 21.80us)        11
       nlp_g  | 106.00us (  9.64us) 107.15us (  9.74us)        11
    nlp_grad  |  77.00us ( 77.00us

----------- MPC execution -----------
Predicted position of cube [0.49999395874039, 0.4363457203901, 0.0849899767977]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 213.00us ( 19.36us) 211.97us ( 19.27us)        11
       nlp_g  |  98.00us (  8.91us)  96.99us (  8.82us)        11
    nlp_grad  |  77.00us ( 77.00us)  76.10us ( 76.10us)         1
  nlp_grad_f  | 568.00us ( 47.33us) 571.56us ( 47.63us)        12
  nlp_hess_l  |   2.89ms (288.60us)   2.89ms (289.27us)        10
   nlp_jac_g  | 209.00us ( 17.42us) 208.96us ( 17.41us)        12
       total  |  25.57ms ( 25.57ms)   9.61ms (  9.61ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999393898593, 0.4363457754571, 0.08498997488403]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 245.00us ( 22.27us) 245.28us ( 22.30us)        11
       nlp_g  | 103.00us (  9.36us) 102.03us (  9.28us)        11
    nlp_grad  |  96.00us ( 96.00us)  

----------- MPC execution -----------
Predicted position of cube [0.49999393820591, 0.43634577709256, 0.084989975629933]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.39ms (126.45us) 252.10us ( 22.92us)        11
       nlp_g  | 104.00us (  9.45us) 104.02us (  9.46us)        11
    nlp_grad  |  95.00us ( 95.00us)  95.13us ( 95.13us)         1
  nlp_grad_f  |   2.33ms (194.58us) 665.77us ( 55.48us)        12
  nlp_hess_l  |   7.84ms (783.90us)   3.36ms (335.83us)        10
   nlp_jac_g  | 507.00us ( 42.25us) 239.56us ( 19.96us)        12
       total  |  44.59ms ( 44.59ms)  11.02ms ( 11.02ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999393820591, 0.43634577709256, 0.084989975629934]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 247.00us ( 22.45us) 247.75us ( 22.52us)        11
       nlp_g  | 106.00us (  9.64us) 106.57us (  9.69us)        11
    nlp_grad  |  93.00us ( 93.00

In [38]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation


0.014474153518676758

t_mpc=50[ms], solver_time=14.4[ms]
--> Realtime loop is available